In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.show_dimensions", False)
pd.set_option("display.float_format", "{:4.2g}".format)

## 5.2 索引存取

`Series` 和 `DataFrame` 提供了豐富的索引存取方法，除了直接使用 `[]` 運算子之外，還可以使用 `.loc[]`, `.iloc[]`, `.at[]`, `.iat[]`, `ix[]` 等存取器存取其中的元素。

表 5-2 DataFrame 物件的各種存取方法
|方法 |說明 |
|-----|-----|
|`[col_label]` |以 單一標籤 作為 索引，取得與標籤對應的列，傳回 `Series` 物件 |
|`[col_labels]` |以 標籤列表 作為 索引，取得對應的多個列，傳回 `DataFrame` 物件 |
|`[row_slice]` |整數切片 或 標籤切片，獲得指定範圍之內的行 |
|`[row_bool_array]` |選擇布林陣列中 `True` 對應的行 |
|`.get(col_label, default)` |與字典的 `get()` 方法的用法相同 |
|`.at[index_label, col_label]` |選擇 行(row)標籤 和 列(column)標籤 對應的值，傳回單一元素 |
|`.iat[index, col]` |選擇 行(row)編號 和 列(column)編號 對應的值，傳回單一元素 |
|`.loc[index, col]` |透過單一標籤值、標籤清單、標籤陣列、布林陣列、標籤切片 等 選擇指定行與列上的資料 |
|`.iloc[index, col]` |透過單一整數值、整數清單、整數陣列、布林陣列、整數切片 等 選擇指定行與列上的資料 |
|`.ix[index, col]` |同時擁有 `.loc[]` 和 `.iloc[]` 的功能，既可以使用標籤索引也可以使用整數索引 |
|`.lookup(row_labels, col_labels)` |選擇行標籤列表與列標籤列表中每對標籤對應的元素值 |
|`.get_value(row_label, col_label)` |與 `.at[]` 的功能類似，不過速度更快 |
|`.query()` |透過運算式選擇滿足條件的行 |
|`.head()` |取得頁首 N 行資料 |
|`.tail()` |取得尾部 N 行資料 |

In [2]:
np.random.seed(42)
df = pd.DataFrame(np.random.randint(0, 10, (5, 3)), 
                  index=["r1", "r2", "r3", "r4", "r5"], 
                  columns=["c1", "c2", "c3"])

### `[]`運算符號

透過 `[]` 運算符號對 `DataFrame` 物件進行存取時，支援以下 5 種索引物件：
- 單一索引標籤(`df[2]`)：取得標籤對應的列(column)，傳回一個 `Series` 物件。
- 多個索引標籤(`df[[1,2,3]]`)：取得以 清單、陣列(注意不能是元組)表示的多個標籤對應的列，傳回一個 `DataFrame` 物件。
- 整數切片(`df[1:4]`)：以整數索引取得切片對應的行。
- 標籤切片(`df["r2":"r4"]`)：當使用標籤作為切片時包含終值。
- 布林陣列(`df[df.c1 > 4]`)：取得陣列中 `True` 對應的行。
- 布林 `DataFrame`(`df[df > 2]`)：將 `DataFrame` 物件中 `False` 對應的元素設定為 `NaN`。

In [3]:
# %C 5 df; df[2:4]; df["r2":"r4"]
print(df)
print("-"*20)
print(df[2:4])
print("-"*20)
print(df["r2":"r4"])

    c1  c2  c3
r1   6   3   7
r2   4   6   9
r3   2   6   7
r4   4   3   7
r5   7   2   5
--------------------
    c1  c2  c3
r3   2   6   7
r4   4   3   7
--------------------
    c1  c2  c3
r2   4   6   9
r3   2   6   7
r4   4   3   7


`df.c1 > 4` 是一個布林序列，因此 `df[df.c1 > 4]` 獲得該序列中 `True` 對應的行。

`df > 2` 是一個布林 `DataFrame` 物件，`df[df > 2]` 將其中 `False` 對應的元素置換為 `NaN`:

In [4]:
# %C 5 df[df.c1 > 4]; df[df > 2]
print(df[df.c1 > 4])
print("-"*20)
print(df[df > 2])

    c1  c2  c3
r1   6   3   7
r5   7   2   5
--------------------
     c1   c2  c3
r1    6    3   7
r2    4    6   9
r3  nan    6   7
r4    4    3   7
r5    7  nan   5


### `.loc[]`和`.iloc[]`存取器

`.loc[]` 的索引物件是一個元組，其中的兩個元素分別與 `DataFrame` 的兩個軸相對應。若索引不是元組，則該索引對應第 0 軸，`:`對應第 1 軸。每個軸的索引物件都支援單一標籤、標籤列表、標籤切片以及布林陣列。

`df.loc["r2"]` 獲得 "r2" 對應的行，它傳回一個 `Series` 物件。

`df.loc["r2", "c2"]` 獲得 "r2" 行 "c2" 列的元素，它傳回單一元素值。

In [5]:
# %C 5 df.loc["r2"]; df.loc["r2","c2"]
print(df.loc["r2"])
print("-"*20)
print(df.loc["r2", "c2"])

c1    4
c2    6
c3    9
Name: r2, dtype: int32
--------------------
6


`df.loc[["r2","r3"]]` 獲得 "r2" 和 "r3" 對應的行。

`df.loc[["r2","r3"],["c1","c2"]]` 則獲得 "r2" 和 "r3" 行、"c1" 和 "c2" 列上的資料，所得到的資料都是新的 `DataFrame` 物件。

In [6]:
# %C 5 df.loc[["r2","r3"]]; df.loc[["r2","r3"],["c1","c2"]]
print( df.loc[["r2","r3"]] )
print("-"*20)
print( df.loc[["r2","r3"],["c1","c2"]] )

    c1  c2  c3
r2   4   6   9
r3   2   6   7
--------------------
    c1  c2
r2   4   6
r3   2   6


In [7]:
# %C 5 df.loc["r2":"r4", ["c2","c3"]]; df.loc[df.c1>2, ["c1","c2"]]
print( df.loc["r2":"r4", ["c2","c3"]] ) # 第 0 軸的索引為標籤切片
print("-"*20)
print( df.loc[df.c1>2, ["c1","c2"]] )   # 第 0 軸的索引為布林數序列

    c2  c3
r2   6   9
r3   6   7
r4   3   7
--------------------
    c1  c2
r1   6   3
r2   4   6
r4   4   3
r5   7   2


`.iloc[]` 和 `.loc[]` 類似，不過它使用整數索引

In [8]:
# %C 5 df.iloc[2]; df.iloc[[2,4]]; df.iloc[[1,3]]; df.iloc[[1,3],[0,2]]
print( df.iloc[2] )
print("-"*20)
print( df.iloc[[2,4]] )
print("-"*20)
print( df.iloc[[1,3]] )
print("-"*20)
print( df.iloc[[1,3],[0,2]] )

c1    2
c2    6
c3    7
Name: r3, dtype: int32
--------------------
    c1  c2  c3
r3   2   6   7
r5   7   2   5
--------------------
    c1  c2  c3
r2   4   6   9
r4   4   3   7
--------------------
    c1  c3
r2   4   9
r4   4   7


In [9]:
# %C 5 df.iloc[2:4, [0,2]]; df.iloc[df.c1.values>2, [0,1]]
print( df.iloc[2:4, [0,2]] )
print("-"*20)
print( df.iloc[df.c1.values>2, [0,1]] )

    c1  c3
r3   2   7
r4   4   7
--------------------
    c1  c2
r1   6   3
r2   4   6
r4   4   3
r5   7   2


目前己經沒有 `.ix[]` 的功能了

In [20]:
# %C 5 df.ix[2:4, ["c1", "c3"]]; df.ix["r1":"r3", [0, 2]]
print( df[["c1", "c3"]].iloc[2:4] )
print("-"*20)
print( df.iloc[:, [0, 2]].loc["r1":"r3", :] )

    c1  c3
r3   2   7
r4   4   7
--------------------
    c1  c3
r1   6   7
r2   4   9
r3   2   7


### 取得單一值

`.at[]` 和 `.iat[]` 分別使用標籤和整數索引取得單一值，此外 `_get_value()` 與 `.at[]` 類似，不過其執行速度要快一些:

In [22]:
# %C 3 df.at["r2", "c2"]; df.iat[1, 1]; df.get_value("r2", "c2")
print( df.at["r2", "c2"] )
print("-"*20)
print( df.iat[1, 1] )
print("-"*20)
print( df._get_value("r2", "c2") )

6
--------------------
6
--------------------
6


如果希望取得兩個列表中每個標籤所對應的元素，可以使用 `.lookup()`，它傳回一個包含指定元素的陣列

`df.lookup(["r2", "r4", "r3"], ["c1", "c2", "c1"])`

相當於

`df.loc['r2','c1'], df.loc['r4','c2'], df.loc['r3','c1']`

In [17]:
print(df)
print("-"*20)
print( df.lookup(["r2", "r4", "r3"], ["c1", "c2", "c1"]) )

    c1  c2  c3
r1   6   3   7
r2   4   6   9
r3   2   6   7
r4   4   3   7
r5   7   2   5
--------------------
[4 3 2]


### 多級標簽的存取

`.loc[]` 和 `.at[]` 的索引可以指定 多級索引 中 每級索引 上的標籤。這時 多級索引軸 對應的索引是一個 索引元組，該元組中的每個元素與索引中的每級索引對應。若索引不是元組，則將其轉為長度為 1 的元組，若元組的長度比索引的層數少，則在其後面補 slice(None)。

In [18]:
soil_df = pd.read_csv("data/Soils-simple.csv", index_col=[0, 1], parse_dates=["Date"])

In [20]:
soil_df

pH  Dens   Ca  Conduc       Date   Name
Depth Contour                                             
0-10  Depression  5.4  0.98   11     1.5 2015-05-26   Lois
      Slope       5.5   1.1   12       2 2015-04-30    Roy
      Top         5.3     1   13     1.4 2015-05-21    Roy
NaN   Depression  4.9   1.4  7.5     5.5 2015-03-21   Lois
10-30 Slope       5.3   1.3  9.5     4.9 2015-02-06  Diana
NaN   Top         4.8   1.3   10     3.6 2015-04-11  Diana

下面實例中，"10-30" 為第 0 軸的標籤，根據前面的規則，將其轉為 ("10-30", slice(None))，即選擇第 0 級中 "10-30" 對應的行:

In [22]:
# %C soil_df.loc["10-30", ["pH", "Ca"]]
print(soil_df.loc["10-30", ["pH", "Ca"]])

          pH   Ca
Contour          
Slope    5.3  9.5


如果需要選擇第 1 級中 "Top" 對應的行，則需要把 slice(None) 作為第 0 級的索引。

由於 python 中只有直接在 `[]` 中才能使用 `:` 分隔的切片語法，因此這裡使用 `np.s_[]` 物件建立第 0 軸對應的索引：`(slice(None), "Top")` 

In [23]:
# %C soil_df.loc[np.s_[:, "Top"], ["pH", "Ca"]]
print(soil_df.loc[np.s_[:, "Top"], ["pH", "Ca"]])

                pH   Ca
Depth Contour          
0-10  Top      5.3   13
NaN   Top      4.8   10


### `query()`方法

當需要根據一定的條件對行(row)進行過濾時，通常可以先建立一個布林陣列，使用該陣列取得 `True` 對應的行，例如下面的程式獲得 pH 值大於 5、Ca 水準小於 11% 的行。由於 python 中無法自訂 not、and 和 or 等關鍵字的行為，因此需要改用 `~`, `&`, `|` 等位元運算符號。然而這些運算子的優先順序比比較運算子要高，因此需要用括號將比較運算括起來：

In [26]:
soil_df[(soil_df.pH >5) & (soil_df.Ca < 11)]

,,pH,Dens,Ca,Conduc,Date,Name
Depth,Contour,,,,,,
0-10,Depression,5.4,0.98,11,1.5,2015-05-26,Lois
10-30,Slope,5.3,1.3,9.5,4.9,2015-02-06,Diana


使用 `.query()` 可以簡化上述程式：

In [25]:
print( soil_df.query("pH > 5 and Ca < 11") )

                   pH  Dens   Ca  Conduc       Date   Name
Depth Contour                                             
0-10  Depression  5.4  0.98   11     1.5 2015-05-26   Lois
10-30 Slope       5.3   1.3  9.5     4.9 2015-02-06  Diana


`.query()` 的參數是一個運算式字串。其中可以使用 not, and, or 等關鍵字進行大量布林運算，運算式中的變數名稱表示與其對應的列。如果希望在運算式中使用其他全域或局域變數的值，可以在變數名稱之前增加`@`，例如：

In [27]:
#%hide_output
pH_low = 5
Ca_hi = 11
print( soil_df.query("pH > @pH_low and Ca < @Ca_hi") )

                   pH  Dens   Ca  Conduc       Date   Name
Depth Contour                                             
0-10  Depression  5.4  0.98   11     1.5 2015-05-26   Lois
10-30 Slope       5.3   1.3  9.5     4.9 2015-02-06  Diana
